# Linear Regression Model that Accurately Predicts the Price of Apples

<img src="predict.jpeg" height=400>

# Table Of Contents
### 1. [Introduction](#introduction)

    1. Objective
    2. Parameters
    3. Outline
    
### 2. [Importing Data and Plotting](#import)

    1. Import necessary packages
    2. Import the data into a Pandas Dataframe
    3. Show the data
    4. Make ean initial plot of the data

### 3. [Exploratory Data Analysis](#explore)
### 4. [Split Data: Testing and training](#split)
### 5. [Outliers](#outliers)
### 6. [Regression Model](#regress)
    1. Taking estimates
    2. Least squares
    3. sklearn
### 7. [Conclusion](#conclude)

## 1. Introduction <a name="introduction"></a>

### 1.1. Objective 
In this notebook we will design a regression model that will predict the cost of apples based on given parameters.

### 1.2. Parameters
1. Month/Season
2. Distance travelled
3. Supplier cost
4. Grade of apple
5. Demand and Supply
6. Container used?


## 2. Importing Data and Plotting <a name="import"></a>

#### 2.1. Import necessary packages

In [4]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import statsmodels as sm
import sklearn as skl

#### 2.2. Import the data into a Pandas Dataframe

#### 2.3. Show the data 

#### 2.4. Make an initial plot of the data

## 3. Exploratory Data Analysis <a name="explore"></a>
### 3.1. Explore the data shape and types
### Look for null values
Give data descriptions
### Is the data univariate or multivariate?
### Determine kurtosis and skew
### Consider the distribution of the data
### Look for correlation of multivariate data 
## 4. Split the data between training data and testing data <a name="split"></a>
## Check for linearity, multicollinearity, independence, homoscedasticity, normality
### Homoscedasticity
Do the magnitude of the risiduals increase as the fitted data increases? This will result in a cone shape and that is called heteroscedasticity. We don’t want that.
### Normality
Draw a histogram of the normalized residuals and look for a bell curve around zero.
Draw a QQ plot of the residuals
## 5. Check for outliers in residuals <a name="outliers"></a>
### Plot Cook’s distance
## 6. Build the Regression Model <a name="regress"></a>
Consider a treemodel
### Results if we follow method 1: Taking Estimates
#### Show our calculations
#### Plot our results
#### Assess our results
### Results if we follow method 2: Least Squared Method
#### Show our calculations
#### Plot our results
#### Assess our results
### Results if we build our model using sklearn:
#### Show our calculations
#### Plot our results
#### Assess our results


## 7. Conclusion <a name="conclude"></a>

### What we accomplished. 
### What we learnt.
